## Document Permissions in Azure AI Search

In [10]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import os

load_dotenv(override=True) # take environment variables from .env.

# The following variables from your .env file are used in this notebook
endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
credential = DefaultAzureCredential()
index_name = os.getenv("AZURE_SEARCH_INDEX", "document-permissions-sample")
indexer_name = os.getenv("AZURE_SEARCH_INDEXER", "document-permissions-sample-indexer")
datasource_name = os.getenv("AZURE_SEARCH_DATASOURCE", "document-permissions-sample-datasource")
adls_gen2_account_name = os.getenv("AZURE_STORAGE_ACCOUNT_NAME", "documentpermissionssample")
adls_gen2_container_name = os.getenv("AZURE_STORAGE_CONTAINER_NAME", "documentpermissionssample")
adls_gen2_connection_string = os.environ["AZURE_STORAGE_CONNECTION_STRING"]
token_provider = get_bearer_token_provider(credential, "https://search.azure.com/.default")

In [2]:
from azure.search.documents.indexes.models import SearchField, SearchIndex, PermissionFilter, SearchIndexPermissionFilterOption
from azure.search.documents.indexes import SearchIndexClient

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index = SearchIndex(
    name=index_name,
    fields=[
        SearchField(name="id", type="Edm.String", key=True, filterable=True, sortable=True),
        SearchField(name="oid", type="Collection(Edm.String)", filterable=True, permission_filter=PermissionFilter.USER_IDS),
        SearchField(name="group", type="Collection(Edm.String)", filterable=True, permission_filter=PermissionFilter.GROUP_IDS),
        SearchField(name="metadata_storage_path", type="Edm.String", searchable=True),
        SearchField(name="metadata_storage_name", type="Edm.String", searchable=True)
    ],
    permission_filter_option=SearchIndexPermissionFilterOption.ENABLED
)

index_client.create_or_update_index(index=index)
print(f"Index '{index_name}' created with permission filter option enabled.")

Index 'document-permissions-sample' created with permission filter option enabled.


In [11]:
from azure.search.documents.indexes.models import SearchIndexerDataSourceConnection, SearchIndexerDataSourceType, IndexerPermissionOption, SearchIndexerDataContainer, DataSourceCredentials
from azure.search.documents.indexes import SearchIndexerClient
indexer_client = SearchIndexerClient(endpoint=endpoint, credential=credential)
datasource = SearchIndexerDataSourceConnection(
    name=datasource_name,
    type=SearchIndexerDataSourceType.ADLS_GEN2,
    connection_string=adls_gen2_connection_string,
    container=SearchIndexerDataContainer(name=adls_gen2_container_name),
    indexer_permission_options=[IndexerPermissionOption.GROUP_IDS]
)

indexer_client.create_or_update_data_source_connection(datasource)
print(f"Datasource '{datasource_name}' created with permission filter option enabled.")

Datasource 'document-permissions-sample-datasource' created with permission filter option enabled.


In [ ]:
from azure.search.documents.indexes.models import SearchIndexer, FieldMapping

indexer = SearchIndexer(
    name=indexer_name,
    target_index_name=index_name,
    data_source_name=datasource_name,
    field_mappings=[
        FieldMapping(source_field_name="metadata_group_ids", target_field_name="group"),
        FieldMapping(source_field_name="metadata_user_ids", target_field_name="oid"),
    ]
)

indexer_client.create_or_update_indexer(indexer)
print(f"Indexer '{indexer_name}' created")


HttpResponseError: () Field mapping specifies target field 'metadata_group_ids' that is not present in the target index
Code: 
Message: Field mapping specifies target field 'metadata_group_ids' that is not present in the target index